# Similarity Measures using Linear Algebra

Example applying linear algebra to calculate similarity between entities. Here, we'll use the [Chicago Public Schools School Profile data for 2017-2018](https://data.cityofchicago.org/Education/Chicago-Public-Schools-School-Profile-Information-/w4qj-h7bg). 

Using these data, we can try to determine which high schools are most similar to each other on a set of different variables.

In [195]:
import pandas as pd

df = pd.read_csv('cps_profiles_public_1718.csv')

cols = ['Long_Name', 'Student_Count_Total', 'Student_Count_Low_Income',
        'Student_Count_English_Learners', 'Student_Count_Special_Ed',
        'Student_Count_White', 'Student_Count_Black', 'Student_Count_Hispanic',
        'College_Enrollment_Rate_School']

cps_df = (df.loc[df['Primary_Category'] == 'HS', cols]
          .dropna(0)
          .set_index('Long_Name'))


This dataset has information about each CPS school. We'll focus on looking at a smaller subset of pertinent variables to compare CPS high school similarity on. These variables were chosen based on both non-sparseness of the data as well as 

* Student_Count_Total
* Student_Count_Low_Income
* Student_Count_English_Learners
* Student_Count_Special_Ed
* Student_Count_White
* Student_Count_Black
* Student_Count_Hispanic
* College_Enrollment_Rate_School

In [196]:
cps_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 161 entries, TEAM Englewood Community Academy High School to Hyman G Rickover Naval Academy High School
Data columns (total 8 columns):
Student_Count_Total               161 non-null int64
Student_Count_Low_Income          161 non-null int64
Student_Count_English_Learners    161 non-null int64
Student_Count_Special_Ed          161 non-null int64
Student_Count_White               161 non-null int64
Student_Count_Black               161 non-null int64
Student_Count_Hispanic            161 non-null int64
College_Enrollment_Rate_School    161 non-null float64
dtypes: float64(1), int64(7)
memory usage: 11.3+ KB


## Algorithm Goals

I will write a cosine similarity algorithm that, given 2 vectors of n-dimensions that will compute the direction resemblance between them, returning a value between -1 and 1. But since we're working in positive space where no elements of any vector are < 0,  the cosine will be [0,1] bound. Cosine = 0 is a perpendicular n-dimensional angle representing maximum independence between vectors, while cosine = 1 means a 0 degree angle representing identical orientation or direction of the compared vectors.  

Cosine similarity is useful in situations where you are concerned more about the direction of two objects being compared as opposed to their magnitudes. It also works well on sparse vectors where 0's or missing information may be present. If [similarity in magnitudes between vectors is primarily important](https://www.quora.com/Is-cosine-similarity-effective), than a simple dot product or Euclidean distance calculation may be more appropriate.  

## How to Solve it?

1. write a function cos_similarity(X, n_similar=5)
    * X = the matrix to compare
    * n_similar = returns a dict of the row index : cosine value for n most similar vectors to current vector.

2. write another function to loop over different vectors. For each individual vector, v1:
    * calculate the cosine similarity of v1 to each other vector
    * add these to an empty list
    * identify the index positions of the top_n_similar vectors to v1
    * create dict pairs of index:value for top_n_similar vectors to v1
    * add this dict to an empty dictionary of the index names
    * repeat loop over next vectors, v2 ... v1000

3. capture the output as a dictionary

In [96]:
def cos_similarity(v1, v2):
    """for 2 non-null vectors, v1 and v2, measure their cosine similarity
    which is the ratio of their dot products to their length products.
    
    (v1 dot v2) / (||v1||*||v2||)
    
    notes:
      v1 dot v2 = sum(v1 * v2)
         ||v1|| = sqrt(v1 * v1)
         ||v2|| = sqrt(v2 * v2)
   """
    
    dot_product, v1_length_sq, v2_length_sq = 0, 0, 0
    
    for element in range(len(v1)):
        # take dot product between v1 and v2, i.e. sum of elementwise products.
        dot_product += v1[element] * v2[element]
        # get the squared vector length of each v1, v2.
        v1_length_sq += v1[element] * v1[element]
        v2_length_sq += v2[element] * v2[element] 
    
    length_product = (v1_length_sq ** (1/2)) * (v2_length_sq ** (1/2))
    
    return dot_product / length_product

In [215]:
import numpy as np
import pandas as pd
rows = 10
cols = 5
X = np.matrix(np.random.randint(0,10, size=(rows, cols)))

df = pd.DataFrame(np.matrix(np.random.randint(0,10, size=(rows, cols))),
                 index = ['exam' + str(i) for i in range(1,11)],
                 columns = ['rule' + str(i) for i in range(1,6)]) 

d1 = df.to_dict('split') # https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_dict.html
d1
d1['data']


[[1, 9, 3, 9, 9],
 [6, 9, 5, 3, 9],
 [6, 4, 1, 4, 7],
 [3, 8, 8, 4, 4],
 [5, 9, 3, 8, 4],
 [8, 0, 1, 2, 3],
 [2, 7, 3, 3, 0],
 [5, 8, 3, 2, 7],
 [1, 7, 5, 4, 7],
 [3, 5, 0, 0, 2]]

In [216]:
cos_similarity(d1['data'][0], d1['data'][1])


0.8667922940439875

In [217]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([d1['data'][0]], [d1['data'][1]])

array([[0.86679229]])

In [218]:
def cos_top_n(x, most_similar=5):
    """For each vector in x, calculate cosine similarity with all
    other vectors not including itself. Return the top 5 most 
    similar vectors as a name:value pair."""
    
    data = x['data']
    index = x['index']
    
    results_dict = {}
    
    for v1 in range(len(data)):
        #if v1 == 0: # temporary scaffolding
            comps = {}
            for v_other in range(len(data)):
                if v1 == v_other:
                    continue
                else:
                    comps[index[v_other]] = cos_similarity(data[v1], data[v_other])
            
            # when done comparing v1 to each v_other, identify the top 5. 
            # https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
            sort_most_similar = sorted(comps.items(), key=lambda d: d[1], reverse=True)[:most_similar]
            results_dict[index[v1]] = dict(sort_most_similar)
            
    return results_dict
              
                

In [219]:
cps_parsed_data = cps_df.to_dict('split')


In [226]:
cps_similarity = cos_top_n(cps_parsed_data, 5)

In [222]:
cps_df.loc[['Instituto Health Sciences Career Academy',
            'Instituto - Justice Lozano',
            'ASPIRA Charter School - Early College High School',
            'World Language Academy High School',
            'YCCS-ASPIRA,Antonia Pantoja Alternative HS',
            'Gurdon S Hubbard High School',
            'Marine Leadership Academy at Ames',
            'David G Farragut Career Academy High School',
            'Greater Lawndale High School For Social Justice',
            'Thomas Kelly High School']]

,Student_Count_Total,Student_Count_Low_Income,Student_Count_English_Learners,Student_Count_Special_Ed,Student_Count_White,Student_Count_Black,Student_Count_Hispanic,College_Enrollment_Rate_School
Long_Name,,,,,,,,
Instituto Health Sciences Career Academy,745,693,213,146,2,14,726,60.4
Instituto - Justice Lozano,89,85,21,13,1,0,87,9.1
ASPIRA Charter School - Early College High School,340,316,108,86,5,7,323,55.3
World Language Academy High School,355,333,103,52,3,17,330,58.4
"YCCS-ASPIRA,Antonia Pantoja Alternative HS",152,122,36,30,3,5,142,15.7
Gurdon S Hubbard High School,1705,1463,297,253,31,112,1547,57.4
Marine Leadership Academy at Ames,849,827,158,82,8,17,819,43.8
David G Farragut Career Academy High School,672,567,177,164,5,84,582,39.8
Greater Lawndale High School For Social Justice,304,260,72,58,2,34,267,50.8
